In [ ]:
import os
import pickle
import gzip

# Get the list of first level subdirectories in dir
data_dir = os.path.expanduser('~/data_Quentin/full_experiments/context_change_MEG')
subdirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Initialize an empty dictionary
result = {}

# Loop through each subdirectory
fnbs = []
for subdir in subdirs:
    # Get the path of the behav directory
    behav_dir = os.path.join(subdir, "behav")
    # Get the list of log files in the behav directory
    log_files = [os.path.join(behav_dir, f) for f in os.listdir(behav_dir) if f.endswith(".log")]
    # Find the largest log file by size
    largest_log = max(log_files, key=os.path.getsize)
    # Get the name of the first level subdirectory
    subdir_name = os.path.basename(subdir)
    # Create a sub-dictionary with the logpath and param_path keys and values
    subdict = {"log_path": largest_log, "param_path": largest_log.replace(".log", ".param"),
              'log_size_mb':os.path.getsize(largest_log) / (1024**2)}
    fnbs.append(largest_log.replace(".log", ''))
    # Add the sub-dictionary to the result dictionary with the subdir_name as the key
    result[subdir_name] = subdict

# Print the result dictionary
print(result)

In [ ]:
fnbs

In [ ]:
path_behav

In [ ]:
fnp

In [ ]:
fn

In [ ]:
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
from datetime import datetime
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
from behav_proc import loadTriggerLog
import pandas as pd
from os.path import join as pjoin

dfs_unproc = []
dftriglogs = []
for fnfb in fnbs:
    path_behav = os.path.dirname(fnfb)
    fnb = os.path.basename(fnfb)
    print(fnb)
    fn = fnb + '.log'
    fnp = fnb + '.param'
    
    params, phase2trigger, trigger2phase, CONTEXT_TRIGGER_DICT  = \
        readParamFiles(fnp, path_behav)
    
    #fnf = fnb + '.log'
    #fnp = fnb + '.param'

    with open(pjoin(path_behav,fn), 'r') as f:
         l = f.readline()
         truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time, time_abs')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)
    assert truelen == len(colnames)
    
    dtypes = (int,int,str,str,str,str,int,float,float,float,float,
        float,float, float,float,float,float, float,float,float,
         float)
    ddt = dict(zip(colnames,dtypes))
    ddt['target_coordX'] = str
    ddt['target_coordY'] = str

    nbad = 0  # recompense strip, but in first particiapnt no
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(path_behav,fn), skipfooter = nbad, 
                     on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1',
                    comment='#', dtype=ddt)
    #encoding='latin-1'
    subj_ = fnb.split('_')[0]    
    df['subject'] = subj_
    #dfs += [df]
    
    df['phase'] = df.apply(lambda row: trigger2phase[row['current_phase_trigger']], 1)
    
    fnf = pjoin(path_behav, fnb + '_trigger.log')
    dftriglog = loadTriggerLog(fnf,CONTEXT_TRIGGER_DICT)
    dftriglog['subject'] = subj_
    dftriglogs += [dftriglog]
    
    ma = df['time_abs'].max()
    ma = datetime.fromtimestamp(ma).strftime("%H:%M:%S")
    mi = df['time_abs'].min()
    mi = datetime.fromtimestamp(mi).strftime("%H:%M:%S")
    print(mi,'   ', ma)
    
    # repair bad target inds for phases where target should be
    # meaningful
    bad_tgti_inds = df.loc[(df['tgti_to_show'].\
            isin(['None','-1'] ) ) &\
            (df['phase'] == 'GO_CUE_WAIT_AND_SHOW')].index
    print('len(bad_tgti_inds) =',len(bad_tgti_inds))
    for ind in bad_tgti_inds:
        nextind = df.loc[ind+1, 'tgti_to_show']
        assert nextind in ['0','1','2']
        print( df.loc[ind+1, 'tgti_to_show'] )
        df.loc[ind,'tgti_to_show'] = nextind
        
    bad_tgti_inds2 = df.loc[(df['tgti_to_show'].isin(['None','-1'] ) ) ].index
    print('len(bad_tgti_inds2) =',len(bad_tgti_inds2))
    df.loc[bad_tgti_inds2,'tgti_to_show'] = '-10000'
    df.loc[bad_tgti_inds2,'target_coordX'] = '-10000'
    df.loc[bad_tgti_inds2,'target_coordY'] = '-10000'
    
    df['tgti_to_show'] = df['tgti_to_show'].astype(int)
    #df['target_coordX'] = df['target_coordX'].astype(int)
    #df['target_coordY'] = df['target_coordY'].astype(int)
    
    df_unproc = df
    dfs_unproc.append(df_unproc)

print( 'len = {}, maxtime in min = {}'.format( len(df), df['time'].max() / 60 ) )

In [ ]:
dfall_unproc = pd.concat(dfs_unproc)
dfall_triglog = pd.concat(dftriglogs)

In [ ]:
dirout = '~/current/merr_data/'
dfall_unproc.to_pickle(os.path.join(dirout, "ccMEG_bahav_collected_rowframe.pkl.gz") ,compression='zip')
# Save the result to a compressed pickle file in dirout
#with gzip.open(os.path.join(dirout, "ccMEG_bahav_collected.pkl.gz"), "wb") as f:
#    pickle.dump(result, f)
dfall_triglog.to_pickle(os.path.join(dirout, "ccMEG_bahav_collected_rowtrig.pkl.gz") ,compression='zip')